# KRK Fault SP vs DP

## Run C++ examples

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

#1ms simulation
DURATION=20
STARTTIMEFAULT=10
ENDTIMEFAULT=10.2
TIMESTEP=1e-3
TIMESTEPSTR=$(printf "%1.6f\n" ${TIMESTEP})

SP_SynGenTrStab_KRK_TwoAreaTrafo_SteadyState

In [ ]:
# Read results
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
from datetime import date
import pandas as pd
from collections import defaultdict

%matplotlib widget

In [ ]:
V_nom = 230e3
V_nom_gen = 20e3

work_dir = 'logs/SP_SynGenTrStab_KRK_TwoAreaTrafo_SteadyState_PF/'
log_name = 'SP_SynGenTrStab_KRK_TwoAreaTrafo_SteadyState_PF'

print(work_dir + log_name + '.csv')

ts_pfsimpy = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

results = pd.DataFrame(columns=['Bus', 'V Mag. [V]', 'V rel Ang. [deg]', 'V Ang. [deg]', 'P', 'Q'])

s_prefix = 's_'
v_prefix = 'v_'
mv_nodes = ['v_bus1','v_bus2','v_bus3','v_bus4']

vble_result_columns_abs = defaultdict(lambda: [0, 0, 0, 0, 0])
for column in ts_pfsimpy.keys():
    if column.startswith(v_prefix):
        column_base = column.replace(v_prefix, '')
        if column in mv_nodes:
            vble_result_columns_abs[column_base][0] = "{0:.2f}".format(np.absolute(ts_pfsimpy[column].values[-1])/V_nom_gen)
        else:
            vble_result_columns_abs[column_base][0] = "{0:.2f}".format(np.absolute(ts_pfsimpy[column].values[-1])/V_nom)
        vble_result_columns_abs[column_base][1] = "{0:.2f}".format(np.degrees(np.angle(ts_pfsimpy[column].values[-1])))
        vble_result_columns_abs[column_base][2] = "{0:.2f}".format(np.degrees(np.angle(ts_pfsimpy[column].values[-1])) - 6.8)
    else:
        column_base = column.replace(s_prefix, '')
        vble_result_columns_abs[column_base][3] = "{0:.2e}".format(np.real(ts_pfsimpy[column].values[-1]))
        vble_result_columns_abs[column_base][4] = "{0:.2e}".format(np.imag(ts_pfsimpy[column].values[-1]))

i = 0
for node,node_data in vble_result_columns_abs.items():
    results.loc[i] = [node] + node_data
    i += 1

print(results)

## Results 1ph SP

In [ ]:
work_dir = 'logs/SP_SynGenTrStab_KRK_TwoAreaTrafo_SteadyState_SP/'
log_name = 'SP_SynGenTrStab_KRK_TwoAreaTrafo_SteadyState_SP'
print(work_dir + log_name + '.csv')
ts_sp1ph_TrStab_dl= rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Parameters

In [ ]:
timestep=50e-6;
t_begin=0
t_end=20

begin_idx = int(t_begin/timestep)
end_idx= int(t_end/timestep)

## Generator terminal voltage

In [ ]:
plt.subplot(2, 2, 1)
for name in ['v_trafo15']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N5')
for name in ['v_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N1')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('voltage (kV)')
plt.title('Gen. 1')

plt.subplot(2, 2, 2)
for name in ['v_trafo26']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N6')

for name in ['v_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N2')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('voltage (kV)')
plt.title('Gen. 2')

plt.subplot(2, 2, 3)
for name in ['v_trafo311']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N11')

for name in ['v_gen3']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N3')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('voltage (kV)')
plt.title('Gen. 3')

plt.subplot(2, 2, 4)
for name in ['v_trafo410']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N10')

for name in ['v_gen4']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N4')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('voltage (kV)')
plt.title('Gen. 4')


## Generator terminal Current

In [ ]:
plt.figure()

plt.subplot(2,2,1)
for name in ['i_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N1', linestyle='--')
    
for name in ['i_trafo15']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N5')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('Gen. 1')

plt.subplot(2,2,2)
for name in ['i_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N2', linestyle='--')
    
for name in ['i_trafo26']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N6')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('Gen. 2')

plt.subplot(2,2,3)
for name in ['i_gen3']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N3', linestyle='--')
    
for name in ['i_trafo311']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N11')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('Gen. 3')

plt.subplot(2,2,4)
for name in ['i_gen4']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N4', linestyle='--')
    
for name in ['i_trafo410']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='N10')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('Gen. 4')


In [ ]:
plt.figure()

plt.subplot(2,2,1)
for name in ['i_line56']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 56')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 56')

plt.subplot(2,2,2)
for name in ['i_line67']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 67')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 67')

plt.subplot(2,2,3)
for name in ['i_line1011']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 1011')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 1011')

plt.subplot(2,2,4)
for name in ['i_line910']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 910')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 910')

In [ ]:
plt.figure()
plt.subplot(1,2,1)
for name in ['i_line78_1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 78_1')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 78_1')

plt.subplot(1,2,2)
for name in ['i_line89_1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-3*ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label='i line 89_1')
    
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('current (kA)')
plt.title('i line 89_1')

## Generator electrical & mechanical energy

In [ ]:
plt.figure()
    
plt.subplot(2,2,1)
for name in ['P_elec1']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-6*ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='P_elec G1')
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.title('P_elec G1')

plt.subplot(2,2,2)
for name in ['P_elec2']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-6*ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='P_elec G2')

# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.title('P_elec G2')

plt.subplot(2,2,3)
for name in ['P_elec3']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-6*ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='P_elec G3')
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.title('P_elec G3')

plt.subplot(2,2,4)
for name in ['P_elec4']:    
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], 1e-6*ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='P_elec G4')
# plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.title('P_elec G4')

## Rotor angular speed $\omega_r$

In [ ]:
plt.figure()

plt.subplot(2,2,1)
for name in ['wr_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='G1')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()
plt.tight_layout()
plt.title('$\omega_{r,1}$')

plt.subplot(2,2,2)
for name in ['wr_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='G2')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()
plt.tight_layout()
plt.title('$\omega_{r,2}$')

plt.subplot(2,2,3)
for name in ['wr_gen3']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='G3')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()
plt.tight_layout()
plt.title('$\omega_{r,3}$')

plt.subplot(2,2,4)
for name in ['wr_gen4']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='G4')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()
plt.tight_layout()
plt.title('$\omega_{r,4}$')

## Rotor angle $\delta _r$

In [ ]:
plt.figure()

plt.subplot(2,2,1)
for name in ['delta_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx],
        ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14 - ts_sp1ph_TrStab_dl['deltaref_gen1'].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='SP')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('angle (°)')
plt.grid()
plt.tight_layout()
plt.title('$\delta_1$')

plt.subplot(2,2,2)
for name in ['delta_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx],
        ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14 - ts_sp1ph_TrStab_dl['deltaref_gen2'].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='SP')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('angle (°)')
plt.grid()
plt.tight_layout()
plt.title('$\delta_2$')

plt.subplot(2,2,3)
for name in ['delta_gen3']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx],
        ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14 - ts_sp1ph_TrStab_dl['delta_gen3'].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='SP')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('angle (°)')
plt.grid()
plt.tight_layout()
plt.title('$\delta_3$')

plt.subplot(2,2,4)
for name in ['delta_gen4']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx],
        ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14 - ts_sp1ph_TrStab_dl['deltaref_gen4'].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='SP')
# plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.ylabel('angle (°)')
plt.grid()
plt.tight_layout()
plt.title('$\delta_4$')
